### Task 2

In [ ]:
import random
from collections import deque
from typing import Deque

GAME_SIZE = 6

def alphabeta(
    state: Deque[int],
    alpha: float = float('-inf'),
    beta: float = float('inf'),
    maximizing: bool = True,
    max_score: int = 0,
    min_score: int = 0
):
    if not state:
        return max_score - min_score, None
    
    best_move = None
    if maximizing:
        max_val = float('-inf')

        left = state.popleft()
        v, _ = alphabeta(state, alpha, beta, False, max_score + left, min_score)
        state.appendleft(left)

        if v > max_val:
            max_val = v
            best_move = 0
        alpha = max(alpha, v)
        if beta <= alpha:
            return max_val, best_move
        
        right = state.pop()
        v, _ = alphabeta(state, alpha, beta, False, max_score + right, min_score)
        state.append(right)

        if v > max_val:
            max_val = v
            best_move = -1
        return max_val, best_move
    else:
        min_val = float('inf')

        left = state.popleft()
        v, _ = alphabeta(state, alpha, beta, True, max_score, min_score + left)
        state.appendleft(left)

        if v < min_val:
            min_val = v
            best_move = 0
        beta = min(beta, v)
        if beta <= alpha:
            return min_val, best_move
        
        right = state.pop()
        v, _ = alphabeta(state, alpha, beta, True, max_score, min_score + right)
        state.append(right)

        if v < min_val:
            min_val = v
            best_move = -1
        return min_val, best_move

game = deque(random.randrange(1, GAME_SIZE) for _ in range(GAME_SIZE))
str_cards = lambda: f"[{", ".join(map(str, game))}]"
print("Initial Cards:", str_cards())

max_score = min_score = 0
while True:
    max_idx = alphabeta(game)[1]
    max_pick = game[max_idx]
    max_score += max_pick
    del game[max_idx]

    print(f"Max picks {max_pick}, Remaining Cards: {str_cards()}")
    if not game:
        break

    min_idx = min(range(len(game)), key=game.__getitem__)
    min_pick = game[min_idx]
    min_score += min_pick
    del game[min_idx]

    print(f"Min picks {min_pick}, Remaining Cards: {str_cards()}")
    if not game:
        break
    
print(f"Final Scores - Max: {max_score}, Min: {min_score}")
print("Winner:", "Max" if max_score > min_score else "Draw" if max_score == min_score else "Min")

In [ ]:
import random
from typing import Set, Tuple, List

SHIP_COUNT = 8

class Board:
    def __init__(self, width: int = 10, height: int = 10):
        self.width = width
        self.height = height
        self.section1 = set()
        self.section2 = set()
        self.divider = width // 2
        self.section1_moves = self._init_moves(1)
        self.section2_moves = self._init_moves(2)
        self.destroyed = set()
        self.moves = []
        
    def generate_random_positions(self, count: int) -> None:
        self.section1 = self._generate_section_positions(0, self.divider, count)
        self.section2 = self._generate_section_positions(self.divider, self.width, count)
        
    def _generate_section_positions(self, x_start: int, x_end: int, count: int) -> Set[Tuple[int, int]]:
        positions = set()
        while len(positions) < count:
            x = random.randint(x_start, x_end - 1)
            y = random.randint(0, self.height - 1)
            pos = x, y
            
            if pos not in positions:
                positions.add(pos)
        return positions

    def _init_moves(self, section: int) -> List[Tuple[int, int]]:
        start_x = 0 if section == 1 else self.divider
        end_x = self.divider if section == 1 else self.width

        legal_moves = []
        for i in range(start_x, end_x):
            for j in range(0, self.height):
                legal_moves.append((i, j))
        return legal_moves
    
    def legal_moves(self, section: int):
        if section == 1:
            return self.section1_moves
        elif section == 2:
            return self.section2_moves
        return []

    def destroy(self, x: int, y: int) -> bool:
        if ((x, y) not in self.section1 and (x, y) not in self.section2) or (x, y) in self.destroyed:
            return False
        
        if (x, y) in self.section1:
            self.section1.remove((x, y))
            self.section1_moves.remove((x, y))
            self.moves.append((1, x, y))
        elif (x, y) in self.section2:
            self.section2.remove((x, y))
            self.section2_moves.remove((x, y))
            self.moves.append((2, x, y))

        self.destroyed.add((x, y))

        return True
    
    def undo(self) -> bool:
        if not self.moves:
            return False
        section, x, y = self.moves.pop()
        self.destroyed.remove((x, y))
        if section == 1:
            self.section1.add((x, y))
            self.section1_moves.append((x, y))
        elif section == 2:
            self.section2.add((x, y))
            self.section2_moves.append((x, y))
        return True

    def display(self, section: int = None) -> None:
        print("Board Layout:")
        for y in range(self.height):
            row = []
            for x in range(self.width):
                if (x, y) in self.section1 and (section == 1 or section == None):
                    row.append("1")
                elif (x, y) in self.section2 and (section == 2 or section == None):
                    row.append("2")
                elif (x, y) in self.destroyed:
                    row.append("x")
                else:
                    row.append("·")
            print(" ".join(row))
        section == 1 and print(f"Section 1 positions: {self.section1}")
        section == 2 and print(f"Section 2 positions: {self.section2}")

board = Board(10, 10)

# assuming these are the positions the player selected
random.seed(42)
board.generate_random_positions(SHIP_COUNT)
board.display(1)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9)]
Board Layout:
1 · · 1 · · · · · ·
1 · · · · · · · · ·
· 1 · · · · · · · ·
· 1 1 · · · · · · ·
· · · · · · · · · ·
· · · · · · · · · ·
· · · · · · · · · ·
· · · · · · · · · ·
1 · · · · · · · · ·
1 · · · · · · · · ·
Section 1 positions: {(0, 1), (1, 2), (0, 0), (0, 9), (3, 0), (2, 3), (0, 8), (1, 3)}
